# 🎬 CineBot - NLP Movie Recommendation Chatbot

**Features:** TF-IDF, Cosine Similarity, Sentiment Analysis, 3D UI

In [ ]:
!pip install gradio scikit-learn pandas textblob -q

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob

movies = [
    {'title': 'The Hangover', 'genre': 'Comedy', 'plot': 'Three friends wake up in Vegas with no memory.', 'keywords': 'funny comedy hilarious'},
    {'title': 'The Conjuring', 'genre': 'Horror', 'plot': 'Paranormal investigators help a terrorized family.', 'keywords': 'scary horror ghost'},
    {'title': 'Titanic', 'genre': 'Romance', 'plot': 'A couple falls in love on the ill-fated ship.', 'keywords': 'love romantic emotional'},
    {'title': 'John Wick', 'genre': 'Action', 'plot': 'An ex-hitman seeks revenge.', 'keywords': 'action revenge guns'},
    {'title': 'Interstellar', 'genre': 'Sci-Fi', 'plot': 'Astronauts travel through a wormhole.', 'keywords': 'space science fiction'},
]

class CineBot:
    def __init__(self):
        self.df = pd.DataFrame(movies)
        self.df['features'] = self.df['plot'] + ' ' + self.df['keywords']
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(self.df['features'])
    
    def get_response(self, msg, history):
        if msg.lower() in ['hi', 'hello']: return 'Hey! What movie are you in the mood for?'
        query_vec = self.vectorizer.transform([msg + ' movie'])
        sim = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        idx = sim.argsort()[-1]
        if sim[idx] < 0.03: return 'Try: comedy, horror, romance, action, sci-fi'
        m = self.df.iloc[idx]
        return f"**{m['title']}** ({m['genre']})\n{m['plot']}"

bot = CineBot()
print('CineBot ready!')

In [ ]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft(primary_hue='purple')) as demo:
    gr.HTML('<h1 style="text-align:center">🎬 CineBot AI</h1>')
    chatbot = gr.Chatbot(height=300)
    msg = gr.Textbox(placeholder='Ask for a movie...')
    
    def respond(m, h):
        if not m.strip(): return h, ''
        return h + [(m, bot.get_response(m, h))], ''
    
    msg.submit(respond, [msg, chatbot], [chatbot, msg])

demo.launch(share=True)